In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

In [4]:
# Loading the CSV files
train_df = pd.read_csv('train.csv')

img_size = (224, 224)

# Function to load images
def load_images(df, path):
    images = []
    labels = []

    for idx, row in df.iterrows():
        img_path = f"{path}/{row['id']}.jpg"
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(row['species'])

    return np.array(images), np.array(labels)

# Converting species names to numerical labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_df['species'])
train_labels_one_hot = to_categorical(train_labels_encoded, num_classes=len(label_encoder.classes_))

# Loading training images
train_images, _ = load_images(train_df, 'images')

# Building the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Adjust output layer units


model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels_one_hot, epochs=10, batch_size=32)
model.save('IMG_leaf_classifier_model.h5')
#model.save('IMG_leaf_classifier_model.keras')




# Evaluate the model on the test set
# test_images, _ = load_images(test_df, 'path/to/test/images')
# test_predictions = model.predict(test_images)
# Add code to handle test predictions as needed

Epoch 1/10
31/31 [==============================] - 277s 9s/step - loss: 130.9811 - accuracy: 0.0505
Epoch 2/10
31/31 [==============================] - 226s 7s/step - loss: 3.0061 - accuracy: 0.3131
Epoch 3/10
31/31 [==============================] - 164s 5s/step - loss: 1.3696 - accuracy: 0.6576
Epoch 4/10
31/31 [==============================] - 139s 5s/step - loss: 0.6401 - accuracy: 0.8212
Epoch 5/10
31/31 [==============================] - 203s 7s/step - loss: 0.3056 - accuracy: 0.9182
Epoch 6/10
31/31 [==============================] - 229s 7s/step - loss: 0.2053 - accuracy: 0.9394
Epoch 7/10
31/31 [==============================] - 166s 5s/step - loss: 0.1428 - accuracy: 0.9556
Epoch 8/10
31/31 [==============================] - 135s 4s/step - loss: 0.1156 - accuracy: 0.9677
Epoch 9/10
31/31 [==============================] - 144s 5s/step - loss: 0.0887 - accuracy: 0.9798
Epoch 10/10
31/31 [==============================] - 107s 3s/step - loss: 0.0794 - accuracy: 0.9808


E:\Downloads\ANACONDA3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

In [7]:
# Load the CSV files
test_df = pd.read_csv('test.csv')

# Function to load test images
def load_test_images(df, path):
    images = []

    for idx, row in df.iterrows():
        img_name = row['id'].astype(np.int64)
        img_path = f"{path}/{img_name}.jpg"
        img = load_img(img_path, target_size=(224, 224))
        img_array = img_to_array(img)
        images.append(img_array)

    return np.array(images)


test_images = load_test_images(test_df, 'images')

model = load_model('IMG_leaf_classifier_model.h5')
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
predicted_species = label_encoder.inverse_transform(predicted_classes)
result_df = pd.DataFrame({'id': test_df['id'], 'predicted_species': predicted_species})

# Saving the results to a CSV file
result_df.to_csv('test_predictions.csv', index=False)

19/19 [==============================] - 27s 712ms/step
